# SWOT data acquisition

#### Jonas Felipe Santos de Souza (jonas.ssouza@ufpe.br)

#### Federal University of Pernambuco

#### June 10, 2025

---

## Bibliotecas

<div class="alert alert-block alert-warning"><b>IMPORTANTE:</b> verifica if the libraries below are installed.</div>

In [10]:
from ipyleaflet import Map, DrawControl, GeoJSON, Popup, Rectangle
from shapely.geometry import Polygon, LineString
from IPython.display import display
import matplotlib.pyplot as plt
from ipywidgets import HTML
import geopandas as gpd
from pathlib import Path
import pandas as pd
import earthaccess
import warnings
import zipfile
import time
import glob
import os


In [9]:
!pip install ipyleaflet
!pip install shapely
!pip install geopandas
!pip install earthaccess

In [ ]:
warnings.filterwarnings("ignore")

---

## Directories

In [15]:
# MAIN DIRECTORY
inpath = 'C:/Users/crist/Desktop/Doctorado/paper/SWOT/swot_rivers/'

# Path to save the obtained products
swotpath = f'{inpath}products/' # *.zip

# SWOT ID of river sections (*.csv file)
swot_id = f'{inpath}nodos_valdivia.csv'

# Path to save SWOT data after extraction
swot_data = f'{inpath}valdivia/'

---

## River_SP SWOT database acquisition

La base de datos debe obtenerse de the *hydroweb.next* platform (https://hydroweb.next.theia-land.fr/).

<div class="alert alert-block alert-warning"><b>SKIP THIS STEP IF YOU ALREADY HAVE THE DATABASE YOU ARE INTERESTED IN.</b> </div>

In [ ]:
!pip install py-hydroweb

In [ ]:
pip install -U py-hydroweb

In [ ]:
help_message = """
Download products from your hydroweb.next projects (https://hydroweb.next.theia-land.fr) using the py-hydroweb lib (https://pypi.org/project/py-hydroweb/)
This script is an example tuned for your last hydroweb.next project but feel free to adapt it for future requests.
Follow these steps:
1. If not already done, install py-hydroweb latest version using `pip install -U py-hydroweb` (WARNING: python >= 3.8 is required)
2a. Generate an API-Key from hydroweb.next portal in your user settings
2b. Carefully store your API-Key (2 options):
- either in an environment variable `export HYDROWEB_API_KEY="<your_key_here>"`
- or in below script by replacing <your_key_here>
3. You can change download directory by adding an `output_folder` parameter when calling `submit_and_download_zip` (see below). By default, current path is used.
4. You are all set, run this script `python download_script.py`

For more documentation about how to use the py-hydroweb lib, please refer to https://pypi.org/project/py-hydroweb/.
"""

In [ ]:
import logging
import sys
from datetime import datetime
from importlib.metadata import version
help_message = """
Error: py_hydroweb no está instalado.
Por favor instálalo con:
    pip install py-hydroweb
"""
try:
    import py_hydroweb
except ImportError:
    print(help_message)
    exit(1)

In [ ]:
# Check py-hydroweb version
import logging
from importlib.metadata import version, PackageNotFoundError  # Python 3.8+

latest_version = "1.0.2"
package_name = "py-hydroweb"

try:
    current_version = version(package_name)
    if current_version < latest_version:
        logging.getLogger().warning(f"""\033[33m
/!\\ Consider upgrading {package_name} to {latest_version} using `pip install -U {package_name}`
(Current version: {current_version})
\033[0m""")
    else:
        print(f"{package_name} is up to date (v{current_version})")
except PackageNotFoundError:
    logging.getLogger().warning(f"""\033[31m
/!\\ The package {package_name} is not installed. Install it using `pip install {package_name}`
\033[0m""")


In [ ]:
# Set log config
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

<div class="alert alert-block alert-warning"><b>IMPORTANTE:</b> Verifica in your hydroweb.next account if the <b>API key</b> has been generated and is active.</div>

In [ ]:
# API key
api_hydroweb = "eMwM9406V62a88130Ydx3Y7uWi4ZAtIVsMCPxg7I7AqcHArVMd"

In [ ]:
# Create a client
#  - either using the API-Key environment variable (HYDROWEB_API_KEY)
#client: py_hydroweb.Client = py_hydroweb.Client("https://hydroweb.next.theia-land.fr/api")
#  - or explicitly giving API-Key (comment line above and uncomment line below)
client: py_hydroweb.Client = py_hydroweb.Client("https://hydroweb.next.theia-land.fr/api", 
                                                api_key=api_hydroweb)

In [ ]:
# Initiate a new download basket (input the name you want here)
basket: py_hydroweb.DownloadBasket = py_hydroweb.DownloadBasket("biobio")

In [ ]:
# Add collections in our basket
# Inserte las coordenadas de la región de interés en el campo bbox
# "SWOT_PRIOR_RIVER_DATABASE"
# "SWOT_PRIOR_LAKE_DATABASE"
basket.add_collection("SWOT_PRIOR_RIVER_DATABASE", 
        #bbox=[-41.40, -9.60, -34.74, -7.10])
        bbox=[-74.091797, -38.894373, -70.378418, -36.300877])

In [ ]:
# Do download (input the archive name you want here, and optionally an output folder)
now = datetime.today().strftime("%Y%m%dT%H%M%S")
downloaded_zip_path: str = client.submit_and_download_zip(
    basket,
    zip_filename=f"{inpath}my_hydroweb_data_{now}.zip",
    #, output_folder = "<change_me>"
)

---

## SWOT product search data configuration

In [20]:
# Shapefile with reaches river
# This file should be obtained from the SWOT database at hydroweb.next
shp = f'{inpath}SWOT_PRIOR_RIVER_DATABASE/sa_sword_nodes_hb66_v17b.shp'

In [21]:
# Dados para busca dos produtos SWOT
# 'SWOT_L2_HR_LakeSP_Prior_2.0', 'SWOT_L2_HR_RiverSP_Reach_2.0', 'SWOT_L2_HR_Raster_100m_2.0'
swot_product = 'SWOT_L2_HR_RiverSP_node_2.0'
short_product = 'SWOT_RiverSP' # 'SWOT_LakeSP', 'SWOT_Raster', 'SWOT_RiverSP'
date_start = '2023-01-01'
date_end = '2026-01-01'
granule_product = '*'

# Plot graphs
ifplot = True # True or False

---

## Mapa para seleccionar el área de ​​interest

In [22]:
# Step 1: Create a map centered on the Amazon region
m = Map(center=(-7.900, -35.250), zoom=8, layout={'height': '600px', 'width': '1000px'}, scroll_wheel_zoom=True)

# Global variables to store the drawn polygon, the GeoJSON layer, and the rectangle layer
polygon = None
popup = None
geojson_layer = None
rectangle_layer = None

# Step 2: Function to handle draw events
def handle_draw(target, action, geo_json):
    global polygon, geojson_layer, rectangle_layer
    if action == 'created':
        # Clear existing GeoJSON layer if it exists
        if geojson_layer:
            m.remove_layer(geojson_layer)

        # Clear the existing rectangle layer if it exists
        if rectangle_layer:
            m.remove_layer(rectangle_layer)

        # Clear the existing polygon layer if it exists
        if polygon:
            polygon_layer = GeoJSON(data={'type': 'Feature', 'geometry': polygon.__geo_interface__})
            m.remove_layer(polygon_layer)

        # Capture the polygon geometry
        geometry = geo_json['geometry']
        polygon = Polygon(geometry['coordinates'][0])
        print("Polygon drawn:", polygon)

        # Draw the rectangle on the map
        minx, miny, maxx, maxy = polygon.bounds
        rectangle_layer = Rectangle(bounds=((miny, minx), (maxy, maxx)), color='blue', fill_opacity=0.1)
        m.add_layer(rectangle_layer)

        # Draw the new polygon layer
        polygon_layer = GeoJSON(data={'type': 'Feature', 'geometry': polygon.__geo_interface__})
        m.add_layer(polygon_layer)

        load_shapefile()  # Load the shapefile based on the drawn polygon
    elif action == 'deleted':
        polygon = None
        if geojson_layer:
            m.remove_layer(geojson_layer)
            geojson_layer = None
        if rectangle_layer:
            m.remove_layer(rectangle_layer)
            rectangle_layer = None  # Reset rectangle_layer to None

# Step 3: Load and filter the shapefile based on the drawn polygon
def load_shapefile():
    global polygon, geojson_layer
    if polygon:
        try:
            gdf = gpd.read_file(shp)
            print(f"Shapefile carregado com sucesso: {len(gdf)} registros encontrados.")
            
            gdf = gdf.to_crs(epsg=4326)
            filtered_gdf = gdf[gdf.intersects(polygon)]
            print(f"{len(filtered_gdf)} registros encontrados dentro da área desenhada.")
            
            geojson_data = filtered_gdf.__geo_interface__
            
            geojson_layer = GeoJSON(data=geojson_data, style={'color': 'green', 'opacity': 0.8, 'weight': 2})

            # Add hover event to display 'lake_if' in a pop-up
            def on_hover(event, feature, **kwargs):
                global popup
                lake_id = feature['properties'].get('reach_id', 'N/A')

                geom = feature['geometry']
                if geom['type'] == 'Polygon' or geom['type'] == 'MultiPolygon':
                    coords = Polygon(geom['coordinates'][0]).centroid.coords[0]
                elif geom['type'] == 'LineString':
                    coords = LineString(geom['coordinates']).centroid.coords[0]
                else:
                    coords = geom['coordinates']

                # Create content for the popup with a selectable 'lake_id'
                html_content = HTML(f'<div style="font-size: 14px;"><b>Reach ID:</b> <span style="user-select: text;">{lake_id}</span></div>')

                # Remove the previous popup if it exists
                if popup:
                    m.remove_layer(popup)

                # Create a new popup with the lake_id and add it to the map
                popup = Popup(
                    location=(coords[1], coords[0]),  # Coordinates (lat, lon)
                    child=html_content,
                    close_button=False,
                    auto_close=False,
                    close_on_escape_key=False
                )
                m.add_layer(popup)

            geojson_layer.on_hover(on_hover)

            m.add_layer(geojson_layer)
        
        except Exception as e:
            print(f"Erro ao carregar o shapefile: {e}")

# Step 4: Add drawing tools to the map, allowing only rectangles
draw_control = DrawControl(
    rectangle={'shapeOptions': {'color': '#0000FF'}},  
    polyline={}, polygon={}, circle={}, marker={}, circlemarker={}
)
draw_control.on_draw(handle_draw)
m.add_control(draw_control)

# Step 5: Display the map


In [23]:
display(m)


Map(center=[-7.9, -35.25], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

---

## Inicio de sesión y búsqueda de productos SWOT in the *EarthData* database

Necesitas una cuenta de EarthData (https://urs.earthdata.nasa.gov/).

<div class="alert alert-block alert-warning"><b>IMPORTANTE:</b> Por favor make sure your EarthData account login and password are correct. </div>

In [ ]:
polygon.bounds

In [3]:
# Comprueba si se ha dibujado el polígono
if 'polygon' in globals():   
    # Earthdata login
    earthaccess.login()
    
    # Obtener datos dentro de los límites del polígono
    results = earthaccess.search_data(short_name = swot_product,
                                      temporal = (date_start, date_end),
                                      #granule_name=granule_product,
                                      bounding_box=(polygon.bounds))
    
    # Mostrar los granules encontrados
    items = [item['meta']['native-id'] for item in results]
    #print(f"Granules encontrados: {items}")
else:
    print("No se dibujaron polígonos.")

# Display the granules found
print(len(items))
print(items)

No se dibujaron polígonos.


NameError: name 'items' is not defined

---

## Descarga de datos SWOT

In [ ]:
# Robust download with retries for transient network errors (IncompleteRead / ChunkedEncodingError)
def _granule_zip_name(granule):
    try:
        return f"{granule['meta']['native-id']}.zip"
    except Exception:
        return None


def _is_valid_zip(path):
    if (not path.exists()) or path.stat().st_size == 0:
        return False
    try:
        with zipfile.ZipFile(path, 'r') as zf:
            return zf.testzip() is None
    except Exception:
        return False


def download_file_with_retries(granule, download_path, max_retries=5, base_wait=5):
    download_dir = Path(download_path)
    download_dir.mkdir(parents=True, exist_ok=True)

    expected_name = _granule_zip_name(granule)
    expected_file = download_dir / expected_name if expected_name else None

    for attempt in range(1, max_retries + 1):
        try:
            earthaccess.download(granule, str(download_dir))

            # Prefer expected file name when available
            candidate = expected_file
            if candidate is None or not candidate.exists():
                matches = sorted(download_dir.glob('SWOT*.zip'), key=os.path.getmtime)
                candidate = matches[-1] if matches else None

            if candidate and _is_valid_zip(candidate):
                print(f"✅ Download OK: {candidate.name}")
                return True

            if candidate and candidate.exists():
                candidate.unlink(missing_ok=True)
            raise IOError("Downloaded file is missing, incomplete, or corrupted.")

        except Exception as e:
            print(f"⚠️ Attempt {attempt}/{max_retries} failed for {expected_name or 'granule'}: {e}")
            # Remove partial target file before retrying
            if expected_file and expected_file.exists():
                expected_file.unlink(missing_ok=True)

            if attempt < max_retries:
                wait_s = base_wait * attempt
                time.sleep(wait_s)
            else:
                print(f"❌ Failed after {max_retries} attempts: {expected_name or 'granule'}")
                return False


# Download files with retries
file_urls = results[:500]
failed = []
for granule in file_urls:
    ok = download_file_with_retries(granule, swotpath)
    if not ok:
        failed.append(_granule_zip_name(granule) or str(granule))

if failed:
    print(f"\nFailed downloads: {len(failed)}")
    for f in failed[:20]:
        print(' -', f)

# Check the most recent file in the download directory
files = glob.glob(swotpath + 'SWOT*.zip')
try:
    file = max(files, key=os.path.getmtime)
    print(f"\nThe most recent file is: {file}")
except ValueError:
    print("\nNo files were downloaded.")


---

## Captura de datos de interés from downloaded SWOT products

<div class="alert alert-block alert-warning"><b>IMPORTANTE:</b> Es útil tener un archivo *.csv with data on the sections of the rivers of interest (name, reaches SWOT ID etc.).</div>

In [ ]:
# *.csv file with SWOT IDs of rivers NODES
nodes_df = pd.read_csv(swot_id, sep=';', decimal=',')
if 'node ID' not in nodes_df.columns:
    raise ValueError("El CSV debe tener una columna 'node ID'.")
nodes_df['node ID'] = nodes_df['node ID'].astype(str)
nodes_df.head()

In [ ]:
import os
import zipfile
import shutil
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from tempfile import mkdtemp  # <--- ESTA LÍNEA


In [ ]:
import os, zipfile
from tempfile import TemporaryDirectory
from collections import defaultdict
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

# --- Parámetros ---
zip_dir     = swotpath          # carpeta con los .zip de SWOT
output_dir  = swot_data         # carpeta de salida
short_product = "Node"          # etiqueta para el archivo de salida
ifplot      = True              # True = graficar, False = solo exportar
valid_node_q = [0, 1]           # calidades válidas

os.makedirs(output_dir, exist_ok=True)

# --- Lista de node_id a procesar (tal cual vienen en tu CSV/DataFrame) ---
nodes_order = list(nodes_df['node ID'])
nodes_set   = set(nodes_order)

# --- Acumulador por node_id ---
# NodeID -> [(Date, wse, wse_u, width, slope, slope_u, width_u, area_total, area_tot_u, area_detct, area_det_u, node_dist), ...]
accum = defaultdict(list)

# --- Recorre cada ZIP una sola vez ---
for filename in os.listdir(zip_dir):
    if not (filename.lower().endswith(".zip") and "node" in filename.lower()):
        continue
    zip_path = os.path.join(zip_dir, filename)

    try:
        with zipfile.ZipFile(zip_path, 'r') as z, TemporaryDirectory(prefix="swot_nodes_") as tmp_dir:
            z.extractall(tmp_dir)

            # Busca .shp (incluye subcarpetas)
            shp_files = []
            for root, _, files in os.walk(tmp_dir):
                shp_files += [os.path.join(root, f) for f in files if f.lower().endswith(".shp")]
            if not shp_files:
                continue

            for shp_path in shp_files:
                try:
                    gdf = gpd.read_file(shp_path)
                except Exception as e:
                    print(f"[WARN] No se pudo leer {shp_path}: {e}")
                    continue

                # Mapeo case-insensitive
                cols = {c.lower(): c for c in gdf.columns}
                time_col      = cols.get('time_str') or cols.get('time')
                node_id_col   = cols.get('node_id')
                node_q_col    = cols.get('node_q')
                wse_col       = cols.get('wse')
                wse_u_col     = cols.get('wse_u')
                width_col     = cols.get('width')
                slope_col     = cols.get('slope')
                slope_u_col   = cols.get('slope_u')
                width_u_col   = cols.get('width_u') or cols.get('wifth_u')
                area_total_col= cols.get('area_total')
                area_tot_u_col= cols.get('area_tot_u')
                area_detct_col= cols.get('area_detct')
                area_det_u_col= cols.get('area_det_u')
                node_dist_col = cols.get('node_dist')

                if not (node_id_col and node_q_col and wse_col and time_col):
                    continue

                # Filtra temprano por tus node_id y calidad válida
                mask = gdf[node_id_col].isin(nodes_set) & gdf[node_q_col].isin(valid_node_q)

                # Columnas a conservar
                keep_cols = [node_id_col, time_col, wse_col]
                optional_cols = [
                    wse_u_col, width_col, slope_col, slope_u_col, width_u_col,
                    area_total_col, area_tot_u_col, area_detct_col, area_det_u_col, node_dist_col
                ]
                keep_cols += [c for c in optional_cols if c]

                sub = gdf.loc[mask, keep_cols].copy()
                if sub.empty:
                    continue

                # Limpia y acumula
                sub["Date"] = pd.to_datetime(sub[time_col], errors="coerce")
                sub["wse"] = pd.to_numeric(sub[wse_col], errors="coerce")

                def add_numeric_col(df, out_name, in_name):
                    if in_name:
                        df[out_name] = pd.to_numeric(df[in_name], errors="coerce")
                    else:
                        df[out_name] = pd.NA

                add_numeric_col(sub, "wse_u", wse_u_col)
                add_numeric_col(sub, "width", width_col)
                add_numeric_col(sub, "slope", slope_col)
                add_numeric_col(sub, "slope_u", slope_u_col)
                add_numeric_col(sub, "width_u", width_u_col)
                add_numeric_col(sub, "area_total", area_total_col)
                add_numeric_col(sub, "area_tot_u", area_tot_u_col)
                add_numeric_col(sub, "area_detct", area_detct_col)
                add_numeric_col(sub, "area_det_u", area_det_u_col)
                add_numeric_col(sub, "node_dist", node_dist_col)

                sub = sub.dropna(subset=["Date", "wse"])

                for row in sub[[
                    node_id_col, "Date", "wse", "wse_u", "width", "slope", "slope_u", "width_u",
                    "area_total", "area_tot_u", "area_detct", "area_det_u", "node_dist"
                ]].itertuples(index=False, name=None):
                    nid, dt, wse, wse_u, width, slope, slope_u, width_u, area_total, area_tot_u, area_detct, area_det_u, node_dist = row
                    accum[nid].append((dt, wse, wse_u, width, slope, slope_u, width_u, area_total, area_tot_u, area_detct, area_det_u, node_dist))

    except zipfile.BadZipFile:
        print(f"[WARN] ZIP corrupto o inválido: {zip_path}")
    except Exception as e:
        print(f"[WARN] Error procesando {zip_path}: {e}")

# --- Exporta y grafica SOLO los node_id de nodes_df (en su orden) ---
for node_id in nodes_order:
    pairs = accum.get(node_id, [])
    if not pairs:
        print(f"No data found for node_id {node_id}.")
        continue

    pairs.sort(key=lambda x: x[0])  # ordenar por fecha
    df = pd.DataFrame(
        pairs,
        columns=["Date", "wse", "wse_u", "width", "slope", "slope_u", "width_u", "area_total", "area_tot_u", "area_detct", "area_det_u", "node_dist"]
    )

    output_csv = os.path.join(output_dir, f"{node_id}_{short_product}.csv")
    df.to_csv(output_csv, sep=',', decimal='.', encoding='utf-8', index=False)
    print(f"Data saved to {output_csv}")

    if ifplot:
        # Gráfico se mantiene SOLO con WSE
        plt.figure(figsize=(10, 6))
        plt.plot(df["Date"], df["wse"], marker='o', linestyle='-')
        plt.xlabel('Date')
        plt.ylabel('WSE (m) EGM08')
        plt.title(f'WSE over Time for Node ID: {node_id}')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
        plt.close()



In [ ]:
import os

print(os.getcwd())  # Muestra la carpeta actual de trabajo
